<a href="https://colab.research.google.com/github/embordin/WorkoutAndBodyMetrics/blob/main/BIA_RawData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
from datetime import datetime, timedelta
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.subplots as sp
import statsmodels
sns.set_theme(style="whitegrid")

In [42]:
pasta_raiz = '/content/drive/MyDrive/EMBordin_Health/'

In [43]:
dates= ['Date']
df_metric = pd.read_excel(f'{pasta_raiz}/RawData_BIA.xlsx', date_parser=dates, sheet_name='metric')

df_metric['FatWater_%']=df_metric['Body_Fat_%']+df_metric['Body_Water_%']
#df_metric=df_metric.query("Date > '06-27-2023'")

df_metric.head(5)

,Date,Sleep_Hours,Time_In_Deep_Sleep,Time_In_Light_Sleep,Time_In_REM_Sleep,Time_Awake,Stress_Level_Max,Stress_Level_AVG,Stress_Qualifier,Body_Battery_Min,Body_Battery_Max,Body_Battery_AVG,BMI,Muscle_Mass_kg,Body_Fat_%,Body_Water_%,Body_Weight_kg,FatWater_%
0,2023-01-07,8.72,0.416667,6.200000,2.100000,1.316667,93.0,27.0,balanced,30.0,87.0,57.0,28.299999,NaN,NaN,0.0,84.83,NaN
1,2023-01-08,6.93,1.183333,5.383333,0.366667,1.266667,93.0,28.0,balanced,32.0,65.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-09,8.08,1.083333,5.850000,1.150000,2.216667,93.0,27.0,balanced,27.0,76.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-10,5.80,1.000000,4.500000,0.300000,1.083333,91.0,28.0,balanced,27.0,56.0,41.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-11,6.93,0.983333,4.050000,1.900000,1.366667,85.0,27.0,balanced,30.0,64.0,45.0,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
#df_metric.apply(lambda x: x.replace('--', 'NaN'))
#df_embordin.apply(lambda x: x.replace('.', ','))

In [45]:
df_metric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 348 non-null    datetime64[ns]
 1   Sleep_Hours          348 non-null    float64       
 2   Time_In_Deep_Sleep   348 non-null    float64       
 3   Time_In_Light_Sleep  348 non-null    float64       
 4   Time_In_REM_Sleep    348 non-null    float64       
 5   Time_Awake           348 non-null    float64       
 6   Stress_Level_Max     346 non-null    float64       
 7   Stress_Level_AVG     346 non-null    float64       
 8   Stress_Qualifier     346 non-null    object        
 9   Body_Battery_Min     347 non-null    float64       
 10  Body_Battery_Max     347 non-null    float64       
 11  Body_Battery_AVG     346 non-null    float64       
 12  BMI                  250 non-null    float64       
 13  Muscle_Mass_kg       245 non-null  

Body Metrics - overview

In [46]:
figures = [
            px.scatter(df_metric, x='Date', y=['Body_Fat_%'],
                       trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace', trendline_color_override='black',
                 opacity=1).update_traces(marker=dict(color='orange')),
            px.scatter(df_metric, x='Date', y=['Body_Water_%'],
                       trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace', trendline_color_override='black',
                 opacity=1).update_traces(marker=dict(color='lightgreen')),
            px.scatter(df_metric, x='Date', y=['Muscle_Mass_kg'],
                       trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace', trendline_color_override='black',
                 opacity=1).update_traces(marker=dict(color='lightblue')),
            px.scatter(df_metric, x='Date', y=['Body_Weight_kg'],
                       trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace', trendline_color_override='black',
                 opacity=1).update_traces(marker=dict(color='salmon')),
            px.scatter(df_metric, x='Date', y=['FatWater_%'],
                        trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace', trendline_color_override='orange',
                opacity=1).update_traces(marker=dict(color='green'))
    ]

fig = make_subplots(rows=len(figures), cols=1, start_cell="bottom-left", subplot_titles=[
                        'Body Fat %', 'Body Water %', 'Body Muscle Mass kg', 'Body Weight kg', 'Body Fat+Water %'])
fig.update_layout(height=600, width=800, title_text="Body Composition Metrics - Moving Average = 3days")

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        fig.add_trace(figure["data"][trace], row=i+1, col=1)

fig.show()

In [47]:
fig= px.scatter(df_metric, x="Date", y=['Body_Fat_%', 'Body_Water_%'],
                title='Body Fat and Body Water - Moving Avg = 3 days', height=600, width=1200,
                 trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace',
                 opacity=1, color_discrete_sequence=px.colors.qualitative.Vivid, template="simple_white",
                 labels={'value':'Body Fat + Body Water %', 'Body Water': 'Body Water %'})

fig.update_layout(title_font_color="Black", font_size=(14),title_x=0.1)
fig.show()

In [48]:
fig= px.scatter(df_metric, x="Date", y=['Body_Weight_kg'],
                title='Body Weight (kg) -  Moving Avg = 3 days', height=600, width=1200,
                 trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace',
                 opacity=1, color_discrete_sequence=px.colors.qualitative.Vivid, template="simple_white",
                 labels={'value':'Body Weight (kg)'})

fig.update_layout(title_font_color="Black", font_size=(14),title_x=0.1)
fig.show()

In [49]:
df_metric1=df_metric.query("Date > '2023-06-26'")

In [50]:
figures = [
            px.scatter(df_metric1, x='Date', y=['Body_Fat_%'],
                       trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace', trendline_color_override='black',
                 opacity=1).update_traces(marker=dict(color='orange')),
            px.scatter(df_metric1, x='Date', y=['Body_Water_%'],
                       trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace', trendline_color_override='black',
                 opacity=1).update_traces(marker=dict(color='lightgreen')),
            px.scatter(df_metric1, x='Date', y=['Muscle_Mass_kg'],
                       trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace', trendline_color_override='black',
                 opacity=1).update_traces(marker=dict(color='lightblue')),
            px.scatter(df_metric1, x='Date', y=['Body_Weight_kg'],
                       trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace', trendline_color_override='black',
                 opacity=1).update_traces(marker=dict(color='salmon')),
            px.scatter(df_metric1, x='Date', y=['FatWater_%'],
                        trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace', trendline_color_override='orange',
                opacity=1).update_traces(marker=dict(color='green'))
    ]

fig = make_subplots(rows=len(figures), cols=1, start_cell="bottom-left", subplot_titles=[
                        'Body Fat %', 'Body Water %', 'Body Muscle Mass kg', 'Body Weight kg', 'Body Fat+Water %'])
fig.update_layout(height=600, width=800, title_text="Body Composition Metrics - Moving Average = 3days")

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        fig.add_trace(figure["data"][trace], row=i+1, col=1)

fig.show()

In [51]:
fig= px.scatter(df_metric1, x="Body_Water_%", y=['Body_Fat_%'],
                title='Body Water % vs Body Fat %', height=600, width=1200,
                 trendline= "ols" , trendline_scope='trace', trendline_color_override='black', color='Body_Weight_kg',
                 opacity=1, color_discrete_sequence=px.colors.qualitative.Vivid, template="simple_white",
                  labels={'value':'Body Fat %', 'Body_Water_%': 'Body Water %'})

fig.update_layout(title_font_color="Black", font_size=(14),title_x=0.1)
fig.show()
results = px.get_trendline_results(fig)
print(results)

results.px_fit_results.iloc[0].summary()

                                      px_fit_results
0  <statsmodels.regression.linear_model.Regressio...


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.968
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                     5195.
Date:                Mon, 29 Jan 2024   Prob (F-statistic):          2.34e-129
Time:                        14:47:36   Log-Likelihood:                 40.450
No. Observations:                 172   AIC:                            -76.90
Df Residuals:                     170   BIC:                            -70.61
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         98.8568      1.120     88.268      0.000      96.646     101.068
x1            -1.3504      0.019    -72.075      0.000      -1.387      -1.313
==============================================================================
Omnibus:                      233.667   Durbin-Watson:                   2.040
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11412.995
Skew:                           5.797   Prob(JB):                         0.00
Kurtosis:                      41.185   Cond. No.                     4.57e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.57e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [161]:
subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.scatter(df_metric1, x="Date", y=['FatWater_%'],
                 trendline= "rolling" ,trendline_options=dict(window=5),trendline_scope='trace',
                 color_discrete_sequence=px.colors.qualitative.Vivid)
fig2 = px.scatter(df_metric1, x="Date", y=['Time_In_Deep_Sleep', 'Time_In_Light_Sleep', 'Time_In_REM_Sleep'],
                  trendline= "rolling" ,trendline_options=dict(window=5),trendline_scope='trace',
                  color_discrete_sequence=px.colors.qualitative.Bold)

fig2.update_traces(yaxis="y2")

subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="Date"
subfig.layout.yaxis.title="Fat+Water %"
subfig.layout.yaxis2.title="Sleep Stage (h)"
subfig.layout.title='Body Fat Water % and Sleep Stage (h) - Moving Avg = 5 days'
subfig.layout.titlefont=dict(family="Arial", size=24, color="gray")
subfig.layout.height=600
subfig.layout.width=1200
subfig.layout.template='simple_white'
subfig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='Lightgray')
subfig.update_yaxes(showgrid=False, gridwidth=1, gridcolor='Lightgray')
subfig.update_yaxes(autorange=True)
#subfig.layout.yaxis2.range=[0,2000]

subfig.show()

In [52]:
fig1= px.scatter(df_metric1, x="Date", y=['FatWater_%', 'Body_Weight_kg'],
                title='Body Fat+Water % - Moving Avg = 3 days', height=600, width=1200,
                 trendline= "rolling" ,trendline_options=dict(window=3),trendline_scope='trace',
                 opacity=1, color_discrete_sequence=px.colors.qualitative.Vivid, template="simple_white",
                 labels={'value':'Body Fat + Body Water %', 'Body Water': 'Body Water %'})
fig1.update_layout(title_font_color="Black", font_size=(14),title_x=0.1)

fig1.show()

In [53]:
fig= px.scatter(df_metric1, x="Body_Weight_kg", y=['Body_Fat_%'],
                title='Body Water % vs Body Fat %', height=600, width=1200,
                 trendline= "ols" , trendline_scope='trace', trendline_color_override='black', color='Body_Weight_kg',
                 opacity=1, color_discrete_sequence=px.colors.qualitative.Vivid, template="simple_white",
                  labels={'value':'Body Fat %', 'Body_Weight_kg': 'Body Weight kg'})

fig.update_layout(title_font_color="Black", font_size=(14),title_x=0.1)
fig.show()
results = px.get_trendline_results(fig)
print(results)

results.px_fit_results.iloc[0].summary()

                                      px_fit_results
0  <statsmodels.regression.linear_model.Regressio...


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     19.40
Date:                Mon, 29 Jan 2024   Prob (F-statistic):           1.87e-05
Time:                        14:47:36   Log-Likelihood:                -247.12
No. Observations:                 172   AIC:                             498.2
Df Residuals:                     170   BIC:                             504.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -15.1102      7.551     -2.001      0.047     -30.015      -0.205
x1             0.4002      0.091      4.404      0.000       0.221       0.580
==============================================================================
Omnibus:                        2.222   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.329   Jarque-Bera (JB):                2.286
Skew:                          -0.261   Prob(JB):                        0.319
Kurtosis:                       2.784   Cond. No.                     8.04e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.04e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [54]:
columns=['Date','Activity Type', 'Distance', 'Calories', 'Time', 'AVG_HR', 'Max_HR', 'Aerobic_TE']
dates= ['Date']
df_workout = pd.read_excel(f'{pasta_raiz}/RawData_BIA.xlsx', date_parser=dates, sheet_name='workout', usecols=columns)

df_workout['Date'] = df_workout['Date'].dt.strftime('%Y-%m-%d')
df_workout['Date'] = pd.to_datetime(df_workout['Date'])

df_workout=df_workout.query("Date > '06-27-2023'")

df_workout.head(5)

,Activity Type,Date,Distance,Calories,Time,AVG_HR,Max_HR,Aerobic_TE
0,Walking,2023-12-29,6.01,435,01:03:41,96,113,1.3
1,Walking,2023-12-28,3.10,222,00:32:40,91,110,0.7
2,Indoor Cycling,2023-12-27,16.88,426,00:31:54,115,130,2.7
3,Walking,2023-12-27,6.03,435,01:02:04,97,113,1.4
4,Indoor Cycling,2023-12-26,22.15,614,00:44:03,122,133,3.2


In [55]:
df_workout.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169 entries, 0 to 168
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Activity Type  169 non-null    object        
 1   Date           169 non-null    datetime64[ns]
 2   Distance       169 non-null    float64       
 3   Calories       169 non-null    int64         
 4   Time           169 non-null    object        
 5   AVG_HR         169 non-null    int64         
 6   Max_HR         169 non-null    int64         
 7   Aerobic_TE     169 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 11.9+ KB


In [56]:
df_fitness=df_metric.merge(df_workout, on='Date', how='inner')
df_fitness

,Date,Sleep_Hours,Time_In_Deep_Sleep,Time_In_Light_Sleep,Time_In_REM_Sleep,Time_Awake,Stress_Level_Max,Stress_Level_AVG,Stress_Qualifier,Body_Battery_Min,...,Body_Water_%,Body_Weight_kg,FatWater_%,Activity Type,Distance,Calories,Time,AVG_HR,Max_HR,Aerobic_TE
0,2023-06-28,9.13,1.066667,6.966667,1.100000,2.100000,96.0,20.0,calm,45.0,...,58.700001,85.13,78.300001,Walking,4.02,273,00:44:29,96,113,1.0
1,2023-06-28,9.13,1.066667,6.966667,1.100000,2.100000,96.0,20.0,calm,45.0,...,58.700001,85.13,78.300001,Running,5.04,423,00:33:05,136,149,3.2
2,2023-06-29,9.63,0.983333,6.250000,2.400000,2.866667,94.0,18.0,calm,43.0,...,58.700001,85.14,78.310001,Treadmill Running,3.01,282,00:22:30,131,143,3.0
3,2023-07-01,7.20,0.216667,4.933333,2.050000,0.866667,90.0,23.0,balanced,40.0,...,58.900002,84.48,78.300002,Treadmill Running,4.21,400,00:30:21,139,150,3.6
4,2023-07-02,8.22,0.566667,5.883333,1.766667,0.750000,99.0,21.0,calm,46.0,...,60.099998,84.87,77.699998,Walking,4.08,278,00:43:27,88,107,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,2023-12-26,7.02,0.133333,5.850000,1.033333,0.683333,99.0,18.0,calm,39.0,...,60.400002,83.83,77.600002,Indoor Cycling,22.15,614,00:44:03,122,133,3.2
165,2023-12-27,7.72,1.083333,5.116667,1.516667,1.266667,99.0,18.0,calm,43.0,...,59.900002,83.37,77.900002,Indoor Cycling,16.88,426,00:31:54,115,130,2.7
166,2023-12-27,7.72,1.083333,5.116667,1.516667,1.266667,99.0,18.0,calm,43.0,...,59.900002,83.37,77.900002,Walking,6.03,435,01:02:04,97,113,1.4
167,2023-12-28,8.82,0.700000,6.233333,1.883333,1.450000,92.0,18.0,calm,45.0,...,59.099998,83.11,78.099998,Walking,3.10,222,00:32:40,91,110,0.7


In [154]:
subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.scatter(df_fitness, x="Date", y=['FatWater_%', 'Body_Weight_kg'],
                 trendline= "rolling" ,trendline_options=dict(window=5),trendline_scope='trace',
                 color_discrete_sequence=px.colors.qualitative.Vivid)
fig2 = px.scatter(df_fitness, x="Date", y=['Calories'],
                  trendline= "rolling" ,trendline_options=dict(window=5),trendline_scope='trace',
                  color_discrete_sequence=px.colors.qualitative.Bold)

fig2.update_traces(yaxis="y2")

subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="Date"
subfig.layout.yaxis.title="Fat+Water % and Weight kg"
subfig.layout.yaxis2.title="Calories"
subfig.layout.title='Body Fat %; Weight and Calories - Moving Avg = 5 days'
subfig.layout.titlefont=dict(family="Arial", size=24, color="gray")
subfig.layout.height=600
subfig.layout.width=1200
subfig.layout.template='simple_white'
subfig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='Lightgray')
subfig.update_yaxes(showgrid=False, gridwidth=1, gridcolor='Lightgray')
#subfig.update_yaxes(autorange=True, range=[0,1000])
subfig.layout.yaxis2.range=[0,2000]

subfig.show()

In [60]:
df_fitness['fatWater_kg']=round(((df_fitness['Body_Fat_%']+df_fitness['Body_Water_%'])/100)*df_fitness['Body_Weight_kg'],2)
df_fitness

,Date,Sleep_Hours,Time_In_Deep_Sleep,Time_In_Light_Sleep,Time_In_REM_Sleep,Time_Awake,Stress_Level_Max,Stress_Level_AVG,Stress_Qualifier,Body_Battery_Min,...,Body_Weight_kg,FatWater_%,Activity Type,Distance,Calories,Time,AVG_HR,Max_HR,Aerobic_TE,fatWater_kg
0,2023-06-28,9.13,1.066667,6.966667,1.100000,2.100000,96.0,20.0,calm,45.0,...,85.13,78.300001,Walking,4.02,273,00:44:29,96,113,1.0,66.66
1,2023-06-28,9.13,1.066667,6.966667,1.100000,2.100000,96.0,20.0,calm,45.0,...,85.13,78.300001,Running,5.04,423,00:33:05,136,149,3.2,66.66
2,2023-06-29,9.63,0.983333,6.250000,2.400000,2.866667,94.0,18.0,calm,43.0,...,85.14,78.310001,Treadmill Running,3.01,282,00:22:30,131,143,3.0,66.67
3,2023-07-01,7.20,0.216667,4.933333,2.050000,0.866667,90.0,23.0,balanced,40.0,...,84.48,78.300002,Treadmill Running,4.21,400,00:30:21,139,150,3.6,66.15
4,2023-07-02,8.22,0.566667,5.883333,1.766667,0.750000,99.0,21.0,calm,46.0,...,84.87,77.699998,Walking,4.08,278,00:43:27,88,107,1.0,65.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,2023-12-26,7.02,0.133333,5.850000,1.033333,0.683333,99.0,18.0,calm,39.0,...,83.83,77.600002,Indoor Cycling,22.15,614,00:44:03,122,133,3.2,65.05
165,2023-12-27,7.72,1.083333,5.116667,1.516667,1.266667,99.0,18.0,calm,43.0,...,83.37,77.900002,Indoor Cycling,16.88,426,00:31:54,115,130,2.7,64.95
166,2023-12-27,7.72,1.083333,5.116667,1.516667,1.266667,99.0,18.0,calm,43.0,...,83.37,77.900002,Walking,6.03,435,01:02:04,97,113,1.4,64.95
167,2023-12-28,8.82,0.700000,6.233333,1.883333,1.450000,92.0,18.0,calm,45.0,...,83.11,78.099998,Walking,3.10,222,00:32:40,91,110,0.7,64.91


In [126]:
subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.scatter(df_fitness, x="Date", y=['Body_Weight_kg', 'fatWater_kg'],
                 trendline= "rolling" ,trendline_options=dict(window=5),trendline_scope='trace',
                 color_discrete_sequence=px.colors.qualitative.Vivid)
fig2 = px.scatter(df_fitness, x="Date", y=['Muscle_Mass_kg'],
                  trendline= "rolling" ,trendline_options=dict(window=5),trendline_scope='trace',
                  color_discrete_sequence=px.colors.qualitative.Bold)

fig2.update_traces(yaxis="y2")

subfig.add_traces(fig.data + fig2.data)
subfig.layout.xaxis.title="Date"
subfig.layout.yaxis.title="Fat+Water kg and Weight kg"
subfig.layout.yaxis2.title="Muscle Mass kg"
subfig.layout.title='Body Fat+Water; Weight and Muscles - Moving Avg = 5 days'
subfig.layout.titlefont=dict(family="Arial", size=24, color="gray")
subfig.layout.height=600
subfig.layout.width=1200
subfig.layout.template='simple_white'
subfig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='Lightgray')
subfig.update_yaxes(showgrid=False, gridwidth=1, gridcolor='Lightgray')
subfig.update_yaxes(autorange=True)

subfig.show()